In [7]:
import pennylane as qml
from pennylane import numpy as np

import scipy as sp

import matplotlib.pyplot as plt

# 2-qubit
dev = qml.device('default.mixed', wires= 2)

# == Hamiltonian setup == 
coeffs = [1]
obs = [qml.PauliZ(0) @ qml.PauliZ(1)]

hamiltonian = qml.Hamiltonian(coeffs, obs)

In [8]:
# == Generate circuit == 

# global var
Paras_global = None

# Entnalger 
@qml.qnode(dev)
def circuit_1(paras):
    theta_init, tau_1, tau_2 = paras
    
    # Stage_1: RY for pi/2
    qml.RY(np.pi/2, wires=0)
    qml.RY(np.pi/2, wires=1)
    
    # Stage_2: Entangler    
    qml.ApproxTimeEvolution(hamiltonian, tau_1, 1)
    qml.RX(theta_init, wires = 0)    
    qml.RX(theta_init, wires = 1)    

    qml.RY(-np.pi/2, wires = 0)    
    qml.RY(-np.pi/2, wires = 1)   

    qml.ApproxTimeEvolution(hamiltonian, tau_2, 1)

    qml.RY(np.pi/2, wires = 0)    
    qml.RY(np.pi/2, wires = 1) 
    
    return qml.density_matrix(wires=[0, 1])

@qml.qnode(dev)
def circuit_2(phi):
    global Paras_global
    density_matrix = circuit_1(Paras_global)
    qml.QubitDensityMatrix(density_matrix, wires=[0, 1])
    
    # Stage_3: Accumulate
    qml.ApproxTimeEvolution(hamiltonian, phi, 1)
    qml.RX(np.pi/2, wires=0)
    qml.RX(np.pi/2, wires=1)
    
    return qml.density_matrix(wires=[0, 1])

In [9]:
# == Cost_function #1 ==
def Cost_function(paras):
    global Paras_global
    # expect: theta_init, tau_1, tau_2 = paras
    Paras_global = paras
    
    phi = np.array([ 2 ])
    
    CFI = qml.qinfo.classical_fisher(circuit_2)(phi[0])
    return -CFI * 1e+30

In [10]:
initial_guess = 1e+5, -1e+6, 1e+6

print('before_optimize:')
print(Cost_function(initial_guess))

After_opt = sp.optimize.minimize(Cost_function, initial_guess, method='SLSQP')

After_opt


before_optimize:
[[-0.52863403]]


 message: Optimization terminated successfully
 success: True
  status: 0
     fun: -1.1541062235682689
       x: [ 4.436e+06 -3.880e+06  1.018e+06]
     nit: 18
     jac: [ 1.912e+07  2.439e+07  1.431e+07]
    nfev: 253
    njev: 18

In [11]:
type(After_opt.fun)

float